In [53]:
using BenchmarkTools
using SpecialFunctions

incbeta_I(x, a, b) = beta_inc(a, b, x)[1]

function ibetar_st(x::T, a, b;
        HUGE_VAL = T(Inf), FDeps = eps(T), rtol = eps(T), MAXITERS=200
    ) where T<:AbstractFloat
    a ≤ 0 && return HUGE_VAL
    if b ≤ 0
        x < 1 && return zero(T)
        x == 1 && return one(T)
        return HUGE_VAL
    end
    x > (a+1)/(a+b+2) && return 1 - ibetar_st(1 - x, b, a; HUGE_VAL, FDeps, rtol, MAXITERS)
    x ≤ FDeps && return zero(T)
    p1 = zero(T)
    q1 = one(T)
    p2 = exp(a*log(x) + b*log(1-x) + loggamma(a+b) - loggamma(a) - loggamma(b)) / a
    q2 = one(T)
    previous = HUGE_VAL
    k = 0
    while k < MAXITERS && !isapprox(p2, previous; rtol)
        previous = p2
        d = -(a+k)*(a+b+k)*x / ((a+2k)*(a+2k+1))
        p1 = p1*d + p2
        q1 = q1*d + q2
        k += 1
        d = k*(b-k)*x /((a+2k-1)*(a+2k))
        p2 = p2*d + p1
        q2 = q2*d + q1
        if q2 ≤ FDeps
            p2 = HUGE_VAL
        else
            p1 /= q2
            q1 /= q2
            p2 /= q2
            q2 = one(T)
        end
    end
    !isapprox(p2, previous; rtol) && error("ibetar did not converge.")
    p2
end

ibetar_st (generic function with 1 method)

In [76]:
a, b = 100.0, 0.01
x = 0.1
@show incbeta_I(x, a, b)
@show ibetar_st(x, a, b);

incbeta_I(x, a, b) = 1.1675552034918687e-104
ibetar_st(x, a, b) = 1.1675552034918835e-104


In [77]:
@btime incbeta_I($x, $a, $b)
@btime ibetar_st($x, $a, $b);

  143.853 ns (0 allocations: 0 bytes)
  149.333 ns (0 allocations: 0 bytes)


In [78]:
@which SpecialFunctions._beta_inc(a, b, x)

_beta_inc(a::Float64, b::Float64, x::Float64) in SpecialFunctions at D:\.julia\packages\SpecialFunctions\hefUc\src\beta_inc.jl:738

In [91]:
@code_warntype SpecialFunctions._beta_inc(a, b, x, 1-x)

MethodInstance for SpecialFunctions._beta_inc(::Float64, ::Float64, ::Float64, ::Float64)
  from _beta_inc(a::Float64, b::Float64, x::Float64, y::Float64) in SpecialFunctions at D:\.julia\packages\SpecialFunctions\hefUc\src\beta_inc.jl:738
Arguments
  #self#::Core.Const(SpecialFunctions._beta_inc)
  a::Float64
  b::Float64
  x::Float64
  y::Float64
Locals
  p::Float64
  q::Float64
  b0::Float64
  n::Int64
  x0::Float64
  y0::Float64
  a0::Float64
  ind::Bool
  lambda::Float64
  epps::Float64
  z::Float64
  @_17::Float64
Body::Tuple{Float64, Float64}
1 ──        Core.NewvarNode(:(b0))
│           Core.NewvarNode(:(n))
│           Core.NewvarNode(:(x0))
│           Core.NewvarNode(:(y0))
│           Core.NewvarNode(:(a0))
│           Core.NewvarNode(:(ind))
│           Core.NewvarNode(:(lambda))
│           Core.NewvarNode(:(epps))
│           Core.NewvarNode(:(z))
│           (p = 0.0)
│           (q = 0.0)
│    %12  = (a < 0.0)::Bool
└───        goto #3 if not %12
2 ──        goto #4
3

In [79]:
epps = max(eps(), 1.0e-15)

1.0e-15

In [81]:
@btime SpecialFunctions.beta_inc_power_series($a, $b, $x, $epps)

  130.464 ns (0 allocations: 0 bytes)


1.1675552034918687e-104

In [83]:
y = 1 - x
lambda = a > b ? (a + b)*y - b : a - (a + b)*x

89.999

In [89]:
SpecialFunctions.beta_inc_asymptotic_symmetric(b, a, lambda, 100.0*eps())

LoadError: AssertionError: a >= 15.0